In [9]:
"""

Integer Multiplication

"""


## imports
import numpy as np

import time


In [14]:

# the 'base' used for 
INT_BASE = 2 ** 16

# decompose into a sequence of machine words
def decompose(a):
    global INT_BASE
    X = []
    d = a

    # while there are more digits
    while d > 0:
        # this would be unneccesary in C, since the number would already be stored
        #   in little endian order
        d, m = divmod(d, INT_BASE)
        X.append(m)

    X.append(0)
    return X

# recompose into a single Python integer
def recompose(X):
    global INT_BASE
    num = 0
    # base ** I
    bi = 1

    # O(n * 1) (also would need to carry)
    for x in X:
        # we use multiplication by a base here; this is O(1) in C though;
        #  because it is a fixed size
        # round to nearest for floating point
        num += bi * int(x + 0.5)
        bi *= INT_BASE

    return num
        
# integer multiplication
def intMM(a, b, stats_dict={}):
    
    # arrays
    aX = decompose(a)
    bX = decompose(b)
    
    # add 0s and pad it
    if len(aX) > len(bX):
        bX = bX + [0] * (len(aX) - len(bX))
    elif len(aX) < len(bX):
        aX = aX + [0] * (len(bX) - len(aX))
        
    assert len(aX) == len(bX)
    
    # zero pad
    aX += [0] * len(aX)
    bX += [0] * len(bX)

    st = time.time()
    
    # frequencies of digits
    faX = np.fft.fft(aX)
    fbX = np.fft.fft(bX)
    
    # pointwise multiplication of frequencies, which convolves the integers
    # think about it:
    #
    #   34
    # x 12
    # ----
    #   68
    #  340
    # ----
    #  408
    #
    # Each of the rows in the second sum is the original sequence (34)
    #   offset by the index of the digit in the second sequence (12),
    #   and scaled by the magnitude
    # This is:
    # sum(sum(A[j] * B[i] * 10**i for j in range(n)) for i in range(n))
    #
    # And so, any particular digit (digit 'd') of the output is:
    # C[d] = sum(A[i - d] * B[i + d] for i in range(n))
    # (not accounting for carry)
    #
    # So, this is a linear sequence that can be modeled as an IR which
    #   the 2 sequences are convolving in
    #
    # Therefore, their product is the convolution of the digits (and carry must be
    #   executed)
    #
    # Since convolve(A, B) == iFFT(FFT(A) .* FFT(B))
    # We can compute it quicker via O(n * logn) FFT algorithm + O(n) multiplication algorithm
    #
    fcX = faX * fbX
    
    # compute inverse FFT
    cX = np.real(np.fft.ifft(fcX))
    
    # store the timing information
    stats_dict['time'] = (time.time() - st) * 1000.0
    
    #print (aX, bX, cX)
    
    # return from bases
    return recompose(cX)

#print (intMM(1234, 5), 1234 * 5)

#print (intMM(1000, 2))

for i in (0, 1, 500, 2320, 19234238):
    for j in (500, 239, 5938283912):
        #print()
        #print (i, j)
        assert (i * j == intMM(i, j))
        #print ((i * j), intMM(i, j))

print ("Integer Multiplication Success!")        
    

Integer Multiplication Success!


In [ ]:

"""

After researching this problem, it turns out to be impossible for quantum computers: https://arxiv.org/abs/quant-ph/0309070


Physically, it cannot carry out the pointwise multiplication required between forward FFTs and inverse FFTs for convolution on the digits

So, this same algorithm cannot be carried out on a quantum computer!


"""

